# Texas Tow Trucks (`.apply` and `requests`)

We're going to scrape some [tow trucks in Texas](https://www.tdlr.texas.gov/tools_search/).

## Import your imports

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import re

/Users/paromasoni/.pyenv/versions/3.8.2/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Search for the TLDR Number `006565540C`, and scrape the information on that company

Using [license information system](https://www.tdlr.texas.gov/tools_search/), find information about the tow truck number above, displaying the

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

If you can't figure a 'nice' way to locate something, your two last options might be:

- **Find a "parent" element, then dig inside**
- **Find all of a type of element** (like we did with `td` before) and get the `[0]`, `[1]`, `[2]`, etc
- **XPath** (inspect an element, Copy > Copy XPath)

These kinds of techniques tend to break when you're on other result pages, but... maybe not! You won't know until you try.

> - *TIP: When you use xpath, you CANNOT use double quotes or Python will get confused. Use single quotes.*
> - *TIP: You can clean your data up if you want to, or leave it dirty to clean later*
> - *TIP: The address part can be tough, but you have a few options. You can use a combination of `.split` and list slicing to clean it now, or clean it later in the dataframe with regular expressions. Or other options, too, probably*

In [2]:
driver = webdriver.Chrome()

In [3]:
driver.get('https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006565540C')

In [4]:
info = driver.find_elements_by_tag_name('td')
name = (info[6].text)
#print (name)

owner = (info[8].text)

phone = (info[10].text)

status = (info[13].text)

addressblock = (info[15].text)
address = addressblock.split('\n\n')[-1]
#print (address)

print (name, owner, phone, status, address)

  To search by tow company or vehicle storage facility, a minimum of the first three letters (characters) of the carrier's name must be entered. Spaces, periods, hyphens, etc. represent characters. The drop-down list provides you the option of searching by 'Customer' or 'DBA' (Doing Business As) name.     City  
   
    
   Show All
   Active
   Administrative Hold
   Enforcement Hold
   Insurance Not Applied
   Expired (less than 1 year)
   
   
   
    
   Show All
   Tow Truck
   Vehicle Storage Facility
   


# Adapt this to work inside of a single cell

Double-check that it works. You want it to print out all of the details.

In [5]:
driver = webdriver.Chrome()
driver.get('https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006565540C')
driver.get('https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006565540C')

#Not sure why but the link only works if you run it two times

info = driver.find_elements_by_tag_name('td')

name = (info[6].text)

owner = (info[8].text)

phone = (info[10].text)

status = (info[13].text)

addressblock = (info[15].text)
address = addressblock.split('\n\n')[-1]


print (name, owner, phone, status, address)

Name:    H & A TOWING LLC Owner/Officer:   HANEEN ABBAS MOHAMMEDAWI / MANAGER Phone:   512-999-8883 Status:  Active Physical:
11710 JOSEPH CLAYTON DR
AUSTIN, TX. 78753


In [6]:
driver = webdriver.Chrome()
driver.get('https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0654479VSF')
driver.get('https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0654479VSF')


In [7]:

#Not sure why but the link only works if you run it two times


info_full = driver.find_elements_by_tag_name('table')[1]
info = info_full.find_elements_by_tag_name('tr')

name = (info[4].text)

owner = (info[5].text)

phone = (info[-3].text)

status = (info[-2].text)

addressblock = (info[-1].text)
address = addressblock.split('\n\n')[-1]

#print (name)
print (name, owner, phone, status, address)

Name:    24/7 TOWING AND RECOVERY LLC DBA:   MIDLAND TOWING AND RECOVERY Owner/Officer:   SCOTT JENSON / VICE PRESIDENT Phone:   4325576733 Certificate Information: Status:  Active Physical:
3601 N COUNTY ROAD 1148
MIDLAND, TX. 79705


# Using .apply to find data about SEVERAL tow truck companies

The file `trucks-subset.csv` has information about the trucks, we'll use it to find the pages to scrape.

### Open up `trucks-subset.csv` and save it into a dataframe

In [8]:
df = pd.read_csv("trucks-subset.csv")
pd.set_option('display.max_colwidth', None)
df

,TDLR Number
0,006565540C
1,0654479VSF
2,006564940C


## Go through each row of the dataset, displaying the URL you will need to scrape for the information on that row

You don't have to actually use the search form for each of these - look at the URL you're on, it has the number in it!

For example, one URL might look like `https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006565540C`.

- *TIP: Use .apply and a function*
- *TIP: You'll need to build this URL from pieces*
- *TIP: You probably don't want to `print` unless you're going to fix it for the next question 
- *TIP: pandas won't showing you the entire url! Run `pd.set_option('display.max_colwidth', None)` to display aaaalll of the text in a cell*

In [9]:
def towtrucks(row):
    #row = df['TDLR Number']
    url = 'https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=' + row
    return url

In [10]:
df.apply(towtrucks)

,TDLR Number
0,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006565540C
1,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0654479VSF
2,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006564940C


In [11]:
df

,TDLR Number
0,006565540C
1,0654479VSF
2,006564940C


### Save this URL into a new column of your dataframe, called `url`

- *TIP: Use a function and `.apply`*
- *TIP: Be sure to use `return`*

In [12]:
def towtrucks(row):
    url = 'https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=' + row
    return url

df['URL'] = df.apply(towtrucks)
df

,TDLR Number,URL
0,006565540C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006565540C
1,0654479VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0654479VSF
2,006564940C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006564940C


## Go through each row of the dataset, printing out information about each tow truck company.

Now will be **scraping** inside of your function.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

Just print it out for now.

- *TIP: use .apply*
- *TIP: You'll be using the code you wrote before, but converted into a function*
- *TIP: Remember how the TDLR Number is in the URL? You don't need to do the form submission if you don't want!*
- *TIP: Make sure you adjust any variables so you don't scrape the same page again and again*

In [13]:
# def towtrucks(URL):
    
#     driver.get(URL)
#     driver.get(URL)
    
#     info = driver.find_elements_by_tag_name('td')
    
#     name = (info[6].text)

#     owner = (info[8].text)

#     phone = (info[10].text)
    
#     status = (info[13].text)
        
#     addressblock = (info[15].text)
#     address = addressblock.split('\n\n')[-1]
    
#     data = {
#         'Name': name,
#         'Owner': owner,
#         'Status': status,
#         'Phone': phone,
#         'Address': address
#     }
    
#     #print (name)
#     return (data)
    
    

In [14]:
def towtrucks2(URL):
    
    driver.get(URL)
    driver.get(URL)
    
    info_full = driver.find_elements_by_tag_name('table')[1]
    info = info_full.find_elements_by_tag_name('tr')

    name = (info[4].text)

    owner = (info[5].text)

    phone = (info[-3].text)

    status = (info[-2].text)

    addressblock = (info[-1].text)
    address = addressblock.split('\n\n')[-1]

    
    #print (name, owner, phone, status, address)

    data = {
        'Name': name,
        'Owner': owner,
        'Status': status,
        'Phone': phone,
        'Address': address
    }
    
    return (data)
    
    

In [15]:
df.URL.apply(towtrucks2)

0                              {'Name': 'Name:    H & A TOWING LLC DBA:   NO DATA', 'Owner': 'Owner/Officer:   HANEEN ABBAS MOHAMMEDAWI / MANAGER', 'Status': 'Certificate Information: Status:  Active', 'Phone': 'Phone:   512-999-8883', 'Address': 'Physical:
11710 JOSEPH CLAYTON DR
AUSTIN, TX. 78753'}
1    {'Name': 'Name:    24/7 TOWING AND RECOVERY LLC DBA:   MIDLAND TOWING AND RECOVERY', 'Owner': 'Owner/Officer:   SCOTT JENSON / VICE PRESIDENT', 'Status': 'Certificate Information: Status:  Active', 'Phone': 'Phone:   4325576733', 'Address': 'Physical:
3601 N COUNTY ROAD 1148
MIDLAND, TX. 79705'}
2                                   {'Name': 'Name:    A & N TOWING LLC DBA:   NO DATA', 'Owner': 'Owner/Officer:   ABDULRAHMAN Z ALABDULLAH / CEO', 'Status': 'Certificate Information: Status:  Active', 'Phone': 'Phone:   2106678546', 'Address': 'Physical:
10838 DEEPWATER BAY
SAN ANTONIO, TX. 78251'}
Name: URL, dtype: object

## Scrape the following information for each row of the dataset, and save it into new columns in your dataframe.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

It's basically what we did before, but using the function a little differently.

- *TIP: Same as above, but you'll be returning a `pd.Series` and the `.apply` line is going to be a lot longer*
- *TIP: Save it to a new dataframe!*
- *TIP: Make sure you change your `df` variable names correctly if you're cutting and pasting - there are a few so it can get tricky*

In [16]:
def towtrucks3(URL):
    
    driver.get(URL)
    driver.get(URL)
    
    info_full = driver.find_elements_by_tag_name('table')[1]
    info = info_full.find_elements_by_tag_name('tr')

    name = (info[4].text)

    owner = (info[5].text)

    phone = (info[-3].text)

    status = (info[-2].text)

    addressblock = (info[-1].text)
    address = addressblock.split('\n\n')[-1]


    data = {
        'Name': name,
        'Owner': owner,
        'Status': status,
        'Phone': phone,
        'Address': address
    }
    
    #print (name)
    return pd.Series(data)
    
    

In [17]:
df2 = df.URL.apply(towtrucks3)
df2

,Name,Owner,Status,Phone,Address
0,Name: H & A TOWING LLC DBA: NO DATA,Owner/Officer: HANEEN ABBAS MOHAMMEDAWI / MANAGER,Certificate Information: Status: Active,Phone: 512-999-8883,"Physical:\n11710 JOSEPH CLAYTON DR\nAUSTIN, TX. 78753"
1,Name: 24/7 TOWING AND RECOVERY LLC DBA: MIDLAND TOWING AND RECOVERY,Owner/Officer: SCOTT JENSON / VICE PRESIDENT,Certificate Information: Status: Active,Phone: 4325576733,"Physical:\n3601 N COUNTY ROAD 1148\nMIDLAND, TX. 79705"
2,Name: A & N TOWING LLC DBA: NO DATA,Owner/Officer: ABDULRAHMAN Z ALABDULLAH / CEO,Certificate Information: Status: Active,Phone: 2106678546,"Physical:\n10838 DEEPWATER BAY\nSAN ANTONIO, TX. 78251"


In [18]:
df2.Name = df2.Name.str.extract(r"Name:([\w\W]*)\sDBA")
df2.Owner = df2.Owner.str.extract(r"Officer:([\w\W]*)\s\/")
df2.Status = df2.Status.str.extract(r"Status:([\w\W]*)")
df2.Phone = df2.Phone.str.extract(r"Phone:([\d\W]*)")
df2.Address = df2.Address.str.extract(r"Physical:([\w\W\d]*)")

df2

,Name,Owner,Status,Phone,Address
0,H & A TOWING LLC,HANEEN ABBAS MOHAMMEDAWI,Active,512-999-8883,"\n11710 JOSEPH CLAYTON DR\nAUSTIN, TX. 78753"
1,24/7 TOWING AND RECOVERY LLC,SCOTT JENSON,Active,4325576733,"\n3601 N COUNTY ROAD 1148\nMIDLAND, TX. 79705"
2,A & N TOWING LLC,ABDULRAHMAN Z ALABDULLAH,Active,2106678546,"\n10838 DEEPWATER BAY\nSAN ANTONIO, TX. 78251"


In [20]:
df2.Address = df2.Address.str.replace('\n', ' ')
df2

,Name,Owner,Status,Phone,Address
0,H & A TOWING LLC,HANEEN ABBAS MOHAMMEDAWI,Active,512-999-8883,"11710 JOSEPH CLAYTON DR AUSTIN, TX. 78753"
1,24/7 TOWING AND RECOVERY LLC,SCOTT JENSON,Active,4325576733,"3601 N COUNTY ROAD 1148 MIDLAND, TX. 79705"
2,A & N TOWING LLC,ABDULRAHMAN Z ALABDULLAH,Active,2106678546,"10838 DEEPWATER BAY SAN ANTONIO, TX. 78251"


### Save your dataframe as a CSV named `tow-trucks-extended.csv`

In [24]:
df2.to_csv("tow-trucks-extended.csv", index=False)

### Re-open your dataframe to confirm you didn't save any extra weird columns

In [25]:
pd.read_csv("tow-trucks-extended.csv")

,Name,Owner,Status,Phone,Address
0,H & A TOWING LLC,HANEEN ABBAS MOHAMMEDAWI,Active,512-999-8883,"11710 JOSEPH CLAYTON DR AUSTIN, TX. 78753"
1,24/7 TOWING AND RECOVERY LLC,SCOTT JENSON,Active,4325576733,"3601 N COUNTY ROAD 1148 MIDLAND, TX. 79705"
2,A & N TOWING LLC,ABDULRAHMAN Z ALABDULLAH,Active,2106678546,"10838 DEEPWATER BAY SAN ANTONIO, TX. 78251"


## Process the entire `tow-trucks.csv` file

We just did it on a short subset so far. Now try it on all of the tow trucks. **Save as the same filename as before**

In [26]:
df3 = pd.read_csv("tow-trucks.csv")
pd.set_option('display.max_colwidth', None)
df3

,TDLR Number
0,006565540C
1,0654479VSF
2,006564940C
3,006542079C
4,0655733VSF
5,006563128C
6,0655400VSF
7,006563749C
8,006539867C
9,006558372C


In [29]:
def towtrucks(row):
    #row = df['TDLR Number']
    url = 'https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=' + row
    return url

df3['url'] = df3.apply(towtrucks)
df3

,TDLR Number,url
0,006565540C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006565540C
1,0654479VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0654479VSF
2,006564940C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006564940C
3,006542079C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006542079C
4,0655733VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0655733VSF
5,006563128C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006563128C
6,0655400VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0655400VSF
7,006563749C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006563749C
8,006539867C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006539867C
9,006558372C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006558372C


In [32]:
def towtrucks4(url):
    
    driver.get(url)
    driver.get(url)
    
    info_full = driver.find_elements_by_tag_name('table')[1]
    info = info_full.find_elements_by_tag_name('tr')

    name = (info[4].text)

    owner = (info[5].text)

    phone = (info[-3].text)

    status = (info[-2].text)

    addressblock = (info[-1].text)
    address = addressblock.split('\n\n')[-1]


    data = {
        'Name': name,
        'Owner': owner,
        'Status': status,
        'Phone': phone,
        'Address': address
    }
    
    #print (name)
    return pd.Series(data)
    

In [33]:
df4 = df3.url.apply(towtrucks4)
df4

,Name,Owner,Status,Phone,Address
0,Name: H & A TOWING LLC DBA: NO DATA,Owner/Officer: HANEEN ABBAS MOHAMMEDAWI / MANAGER,Certificate Information: Status: Active,Phone: 512-999-8883,"Physical:\n11710 JOSEPH CLAYTON DR\nAUSTIN, TX. 78753"
1,Name: 24/7 TOWING AND RECOVERY LLC DBA: MIDLAND TOWING AND RECOVERY,Owner/Officer: SCOTT JENSON / VICE PRESIDENT,Certificate Information: Status: Active,Phone: 4325576733,"Physical:\n3601 N COUNTY ROAD 1148\nMIDLAND, TX. 79705"
2,Name: A & N TOWING LLC DBA: NO DATA,Owner/Officer: ABDULRAHMAN Z ALABDULLAH / CEO,Certificate Information: Status: Active,Phone: 2106678546,"Physical:\n10838 DEEPWATER BAY\nSAN ANTONIO, TX. 78251"
3,Name: AR TOWING LLC DBA: NO DATA,Owner/Officer: TANYA M JOHNSON / PRESIDENT,Certificate Information: Status: Active,Phone: 4693059587,"Physical:\n3014 AURORA MIST DR\nLITTLE ELM, TX. 75068"
4,"Name: BEST IN THE WEST TOWING, LLC DBA: NO DATA",Owner/Officer: DANIEL L NASTANSKY / CEO,Certificate Information: Status: Active,Phone: 4325575551,"Physical:\n2801 N FREMONT AVE\nODESSA, TX. 79764"
5,Name: BOUNCING TOWING LLC DBA: NO DATA,Owner/Officer: JONNY B MMADUEGBUNA / MANAGER,Certificate Information: Status: Active,Phone: 4697357540,"Physical:\n3017 ELMBROOK DR\nGRAND PRAIRIE, TX. 75052"
6,Name: ALL AMERICAN TOWING AND RECOVERY INC DBA: NO DATA,Owner/Officer: MICHAEL THOMAS PHILLIPS / PRESIDENT,Certificate Information: Status: Expired,Phone: 940-627-2800,"Physical:\n2618 S HIGHWAY 287\nDECATUR, TX. 76234"
7,Name: BROTHERS TOWING AND RECOVERY LLC DBA: NO DATA,Owner/Officer: ADOLFO MARTINEZ JR / CEO,Certificate Information: Status: Active,Phone: 9562357894,"Physical:\n199 NORTH RIATA ROAD\nLAREDO, TX. 78043"
8,Name: CITYWIDE AUTO TOWING INC DBA: NO DATA,Owner/Officer: MOHAMMAD ALQARNI / CEO,Certificate Information: Status: Active,Phone: 7132406665,"Physical:\n3410 DUNVALE RD\nHOUSTON, TX. 77063"
9,Name: DAGI TOWING LLC DBA: NO DATA,Owner/Officer: ANDY ZEWDE JORDAN / MANAGER,Certificate Information: Status: Active,Phone: 5129207530,"Physical:\n11108 GEORGIAN DR\nAUSTIN, TX. 78753"


In [34]:
df4.Name = df4.Name.str.extract(r"Name:([\w\W]*)\sDBA")
df4.Owner = df4.Owner.str.extract(r"Officer:([\w\W]*)\s\/")
df4.Status = df4.Status.str.extract(r"Status:([\w\W]*)")
df4.Phone = df4.Phone.str.extract(r"Phone:([\d\W]*)")
df4.Address = df4.Address.str.extract(r"Physical:([\w\W\d]*)")

df4

,Name,Owner,Status,Phone,Address
0,H & A TOWING LLC,HANEEN ABBAS MOHAMMEDAWI,Active,512-999-8883,"\n11710 JOSEPH CLAYTON DR\nAUSTIN, TX. 78753"
1,24/7 TOWING AND RECOVERY LLC,SCOTT JENSON,Active,4325576733,"\n3601 N COUNTY ROAD 1148\nMIDLAND, TX. 79705"
2,A & N TOWING LLC,ABDULRAHMAN Z ALABDULLAH,Active,2106678546,"\n10838 DEEPWATER BAY\nSAN ANTONIO, TX. 78251"
3,AR TOWING LLC,TANYA M JOHNSON,Active,4693059587,"\n3014 AURORA MIST DR\nLITTLE ELM, TX. 75068"
4,"BEST IN THE WEST TOWING, LLC",DANIEL L NASTANSKY,Active,4325575551,"\n2801 N FREMONT AVE\nODESSA, TX. 79764"
5,BOUNCING TOWING LLC,JONNY B MMADUEGBUNA,Active,4697357540,"\n3017 ELMBROOK DR\nGRAND PRAIRIE, TX. 75052"
6,ALL AMERICAN TOWING AND RECOVERY INC,MICHAEL THOMAS PHILLIPS,Expired,940-627-2800,"\n2618 S HIGHWAY 287\nDECATUR, TX. 76234"
7,BROTHERS TOWING AND RECOVERY LLC,ADOLFO MARTINEZ JR,Active,9562357894,"\n199 NORTH RIATA ROAD\nLAREDO, TX. 78043"
8,CITYWIDE AUTO TOWING INC,MOHAMMAD ALQARNI,Active,7132406665,"\n3410 DUNVALE RD\nHOUSTON, TX. 77063"
9,DAGI TOWING LLC,ANDY ZEWDE JORDAN,Active,5129207530,"\n11108 GEORGIAN DR\nAUSTIN, TX. 78753"


In [35]:
df4.Address = df4.Address.str.replace('\n', ' ')
df4

,Name,Owner,Status,Phone,Address
0,H & A TOWING LLC,HANEEN ABBAS MOHAMMEDAWI,Active,512-999-8883,"11710 JOSEPH CLAYTON DR AUSTIN, TX. 78753"
1,24/7 TOWING AND RECOVERY LLC,SCOTT JENSON,Active,4325576733,"3601 N COUNTY ROAD 1148 MIDLAND, TX. 79705"
2,A & N TOWING LLC,ABDULRAHMAN Z ALABDULLAH,Active,2106678546,"10838 DEEPWATER BAY SAN ANTONIO, TX. 78251"
3,AR TOWING LLC,TANYA M JOHNSON,Active,4693059587,"3014 AURORA MIST DR LITTLE ELM, TX. 75068"
4,"BEST IN THE WEST TOWING, LLC",DANIEL L NASTANSKY,Active,4325575551,"2801 N FREMONT AVE ODESSA, TX. 79764"
5,BOUNCING TOWING LLC,JONNY B MMADUEGBUNA,Active,4697357540,"3017 ELMBROOK DR GRAND PRAIRIE, TX. 75052"
6,ALL AMERICAN TOWING AND RECOVERY INC,MICHAEL THOMAS PHILLIPS,Expired,940-627-2800,"2618 S HIGHWAY 287 DECATUR, TX. 76234"
7,BROTHERS TOWING AND RECOVERY LLC,ADOLFO MARTINEZ JR,Active,9562357894,"199 NORTH RIATA ROAD LAREDO, TX. 78043"
8,CITYWIDE AUTO TOWING INC,MOHAMMAD ALQARNI,Active,7132406665,"3410 DUNVALE RD HOUSTON, TX. 77063"
9,DAGI TOWING LLC,ANDY ZEWDE JORDAN,Active,5129207530,"11108 GEORGIAN DR AUSTIN, TX. 78753"


In [36]:
df4.to_csv("tow-trucks-extended.csv", index=False)


In [37]:
pd.read_csv("tow-trucks-extended.csv")

,Name,Owner,Status,Phone,Address
0,H & A TOWING LLC,HANEEN ABBAS MOHAMMEDAWI,Active,512-999-8883,"11710 JOSEPH CLAYTON DR AUSTIN, TX. 78753"
1,24/7 TOWING AND RECOVERY LLC,SCOTT JENSON,Active,4325576733,"3601 N COUNTY ROAD 1148 MIDLAND, TX. 79705"
2,A & N TOWING LLC,ABDULRAHMAN Z ALABDULLAH,Active,2106678546,"10838 DEEPWATER BAY SAN ANTONIO, TX. 78251"
3,AR TOWING LLC,TANYA M JOHNSON,Active,4693059587,"3014 AURORA MIST DR LITTLE ELM, TX. 75068"
4,"BEST IN THE WEST TOWING, LLC",DANIEL L NASTANSKY,Active,4325575551,"2801 N FREMONT AVE ODESSA, TX. 79764"
5,BOUNCING TOWING LLC,JONNY B MMADUEGBUNA,Active,4697357540,"3017 ELMBROOK DR GRAND PRAIRIE, TX. 75052"
6,ALL AMERICAN TOWING AND RECOVERY INC,MICHAEL THOMAS PHILLIPS,Expired,940-627-2800,"2618 S HIGHWAY 287 DECATUR, TX. 76234"
7,BROTHERS TOWING AND RECOVERY LLC,ADOLFO MARTINEZ JR,Active,9562357894,"199 NORTH RIATA ROAD LAREDO, TX. 78043"
8,CITYWIDE AUTO TOWING INC,MOHAMMAD ALQARNI,Active,7132406665,"3410 DUNVALE RD HOUSTON, TX. 77063"
9,DAGI TOWING LLC,ANDY ZEWDE JORDAN,Active,5129207530,"11108 GEORGIAN DR AUSTIN, TX. 78753"
